In [191]:
import numpy as np #importa biblioteca Numpy

In [192]:
class probabilidade_dados: # Cria a classe
  """

  Attributes:
    numero_dados:
    tamanho_dados:
    todos_resultados:
    tamanho_min:
    tamanho_max:
    porcentagem:
    valores_possiveis:
    soma_porcentagem:
  """
  def __init__(self, numero_dados= 1, tamanho_dados= 1, tamanho_min= 0, tamanho_max = (2**64), todos_resultados = True, porcentagem = False, soma_porcentagem= False):
  # Inicia a classe com suas variaveis

    #Inicializa as variaveis com os valores passados ou pré-definidos
    #Transforma todos os numeros em Integer
    self.numero_dados = int(numero_dados) # Quantos dados utilizados
    self.tamanho_dados = int(tamanho_dados) # Tamanho dos dados, 4, 6, 8 lados etc...
    self.tamanho_min = int(tamanho_min) # Tamanho minimo cuja propabilidade queremos avaliar

    #Verifica se a variavel Tamanho Maximo é maior do que o numero maximo alcançavel pelo dado
    #se não for, o Tamanho Maximo se torna o maior numero possivel
    #Tamanho maximo cuja probabilidade queremos avaliar
    #Caso o usuario não passe um parametro esse valor está em um numero absurdo
    #para que entre no if e se torne igual ao maior numero alcançavel
    if tamanho_max >= self.numero_dados*self.tamanho_dados:
      self.tamanho_max = self.numero_dados*self.tamanho_dados #maior numero alcançavel é igual a quantidade de lados de cada dado vezes a quantidade de dados rolados
    else:
      self.tamanho_max = int(tamanho_max)

    # Verifica se queremos todos os resultados
    #Serve para facilitar a apresentação da função, caso o usuario não queira passar todos os parametros
    self.todos_resultados = todos_resultados
    if self.todos_resultados:
      print("Foram pedidos todos os resultados, caso queire um resultado expecifico marque essa flag como False")
      self.tamanho_min = 0
      self.tamanho_max = self.tamanho_dados*self.numero_dados

    self.porcentagem=porcentagem #verifica se queremos o valor rm forma de porcentagem
    self.soma_porcentagem= soma_porcentagem #verifica se queremos a soma da probabilidade de diversos valores

    #função principal, que faz o calculo
    self.calcular_numero_somas(n_min = self.tamanho_min, n_max = self.tamanho_max, porcentagem = self.porcentagem, soma_porcentagem= self.soma_porcentagem)

  def calcular_numero_somas(self, n_min, n_max, porcentagem, soma_porcentagem):

    #vetor que armazena todos os valores possiveis
    #o maior valor possivel é caso a rolagem gere todos os maiores valores em todos os lados
    #o valor então é somado
    #por exemplo, 24 em 3 dados de 8 lados
    valores_possiveis = np.zeros((self.numero_dados*self.tamanho_dados)+1)

    #matriz de colunas quase iguais, caa coluna passa de 1 até o valor maximo do dado
    #e tem linhas iguais a quantidade de dados rolados
    #a ultima linha da matriz, inicializa-se sempre como 0
    #ela existe para identificar quais valores serão utilizados em uma soma
    #por exemplo se a ultima linha for [-1, 0, -3]
    #isso significa que o programa somara os itens [0, 1] + [1, 0] + [2, 3]
    matriz_dados = np.append(np.array(range(1, self.tamanho_dados+1)), 0)
    for i in range(self.numero_dados-1):
      matriz_dados = np.column_stack((matriz_dados, np.append(np.array(range(1, self.tamanho_dados+1)), 0)))

    aux = self.tamanho_dados #auxiliar para facilitar os calculos e a apresentação do programa

    #a primeira soma guiada pela Matriz Dados possiveis é feita antes de entrarmos nos laços de repetição
    valores_possiveis[np.sum(matriz_dados[0,:])] += 1

    continua = True # flag para continuarmos o laço de repetição inicial
    while(continua):
      soma = 0 # variavel soma é zerada
      k = self.numero_dados-1 # variavel k inicia-se com o numero da ultima coluna de Matriz Dados

      while(k>=0):
        #verificamos se o ultimo elemento da coluna avaliada representa o maior valor tiravel pelo dado
        #e assim da ultima linha da parte que desejamos avaliar da Matriz dado
        #para evitar problemas matematicos todos os valores da ultima linha são <=0
        #uma vez que um dado usual só tem lados positivos
        #logo todos os valores da ultima linha, e suas consequentes analises são <=0
        if matriz_dados[aux, k] == (self.tamanho_dados)*(-1):
          #verificamos se TODOS os valores da ultima linha indicam o fim daquela coluna
          #dessa forma teremos terminado a matriz e deveremos parar o laço
          if (matriz_dados[aux, :] == (self.tamanho_dados)*(-1)).all():
            continua = False
            k = -1
          #Caso continuemos os laçoes de repetição e essa coluna tenha chegado ao seu fim
          #Re-inicializamos o ultimo valor da coluna analisada
          #e fazemos a mesma analise na coluna anterior
          else:
            matriz_dados[aux, k] = 0
            k -= 1
        else:
          #Caso aquela coluna não tenha chegado ao seu fim, diminuimos o valor
          #de sua ultima linha em 1, para testes futuros
          matriz_dados[aux, k] -= 1
          k = -5 # k é mudado para um valor < 0, de forma a sairmos do laço de repetição

          #If utilizado para que a existência da ultima linha seja ignorada durante a
          #quantidade de somas
          #So entramos no If se NÃO HOUVER NENHUM valor da igual ao valor maximo
          #negativo alcançavel por ela
          #dessa forma o vator da matriz, que existe para apontar para quais valores
          #do resto da matriz deva ser utilizado, não possa apontar para um valor
          #dentro da linha de controle
          if not((matriz_dados[-1,:] == (self.tamanho_dados)*(-1)).any()):

            #todos os valores apontados pela linha de controle são somados
            for i in range(self.numero_dados):
              soma += matriz_dados[(-1*matriz_dados[aux, i]), i]

            #O vetor valores_possiveis e atualizado para representar que existe mais uma
            #soma que resulte no valor daquele index
            valores_possiveis[soma] += 1

    #print(valores_possiveis)

    #O vetor com os valores é dividido pela quantidade maxima de somas que pode ser calculada como
    # tamanho dos dados ^ numero de dados rolados
    if porcentagem: #verifica se desejamos o valor em porcentagem ou aritmetico
      if soma_porcentagem: #verifica se queremos uma tabela com cada valor ou a sua soma
        #Imprime a soma dos valores pedidos em porcentagem
        print("Soma: %s%%" % np.sum((valores_possiveis[n_min : n_max+1]/(self.tamanho_dados**self.numero_dados))*100))
      else:
        #Imprime a tabela dos valores pedidos em porcentagem
        for i in range(n_min, n_max+1):
          print("%s: %s%%" % (i, (valores_possiveis[i]/(self.tamanho_dados**self.numero_dados))*100))
    else:
      if soma_porcentagem:
        #Imprime a soma dos valores pedidos de forma aritmetica
        print("Soma: %s" % np.sum((valores_possiveis[n_min : n_max+1]/(self.tamanho_dados**self.numero_dados))))
      else:
        #Imprime a tabela dos valores pedidos de forma aritmetica
        for i in range(n_min, n_max+1):
          print("%s: %s" % (i, (valores_possiveis[i]/(self.tamanho_dados**self.numero_dados))))

In [194]:
probabilidade_dados(numero_dados= 3, tamanho_dados=4, tamanho_min= 3, tamanho_max= 9, todos_resultados= False, porcentagem=False, soma_porcentagem= True)

Soma: 0.84375
